In [ ]:
#https://github.com/csauyong/cnn-cifar/blob/main/CIFAR_10.ipynb

# Import PyTorch and other libraries
import torch
import torchvision

# Define the device to use (GPU or CPU)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the CIFAR-10 dataset and split it into train and test sets
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# Define the CNN model with three convolutional layers and a MLP classifier
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, 3) # First convolutional layer with 32 filters of size 3x3
        self.pool = torch.nn.MaxPool2d(2, 2) # Pooling layer with 2x2 window size
        self.conv2 = torch.nn.Conv2d(32, 64, 3) # Second convolutional layer with 64 filters of size 3x3
        self.conv3 = torch.nn.Conv2d(64, 64, 3) # Third convolutional layer with 64 filters of size 3x3
        self.fc1 = torch.nn.Linear(64 * 4 * 4, 256) # First fully-connected layer with 256 nodes
        self.fc2 = torch.nn.Linear(256, 128) # Second fully-connected layer with 128 nodes
        self.fc3 = torch.nn.Linear(128, 10) # Output layer with 10 nodes
        self.relu = torch.nn.ReLU() # ReLU activation function
        self.softmax = torch.nn.Softmax(dim=1) # Softmax activation function

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x))) # Apply first convolutional layer, then ReLU, then pooling
        x = self.pool(self.relu(self.conv2(x))) # Apply second convolutional layer, then ReLU, then pooling
        x = self.relu(self.conv3(x)) # Apply third convolutional layer, then ReLU
        x = x.view(-1, 64 * 4 * 4) # Flatten the feature map
        x = self.relu(self.fc1(x)) # Apply first fully-connected layer, then ReLU
        x = self.relu(self.fc2(x)) # Apply second fully-connected layer, then ReLU
        x = self.softmax(self.fc3(x)) # Apply output layer, then softmax
        return x

model = CNN().to(device) # Create the model and move it to the device

# Define the loss function and the optimizer
criterion = torch.nn.CrossEntropyLoss() # Categorical crossentropy loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01) # SGD optimizer

# Train the model for 20 epochs
for epoch in range(20):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        # Compute the loss
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # Print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

# Test the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))
